## Load keypoints

In [4]:
from pycocotools.coco import COCO
## install coco library https://github.com/pdollar/coco
from collections import namedtuple
import numpy as np
from numpy import ma
import math
import skimage.io as io
import matplotlib.pyplot as plt
import matplotlib
import pylab
import json
import time
from numpy import linalg as LA
pylab.rcParams['figure.figsize'] = (10.0, 8.0)

## load keypoints json
annFile = '/data/datasets/COCO/person_keypoints_trainval2014/person_keypoints_train2014.json' # keypoint file
trainimagepath = '/data/guest_users/liangdong/liangdong/practice_demo/train2014/'             # train image path

coco = COCO(annFile)
cats = coco.loadCats(coco.getCatIds())
catIds = coco.getCatIds(catNms=['person'])
imgIds = coco.getImgIds(catIds=catIds )
jointall=[]

loading annotations into memory...
Done (t=6.37s)
creating index...
index created!


In [5]:
## train ids 
len(imgIds)

45174

## Transform data format from coco to real_time_pose 

In [6]:
count = 0
joint_all = dict()
numimgs = len(imgIds)

for i in range(0, numimgs):
    #print('----image: '+str(imgIds[i])+' ---------------')
    img = coco.loadImgs(imgIds[i])[0]
    annIds = coco.getAnnIds(imgIds=img['id'], catIds=catIds, iscrowd=None)
    anns = coco.loadAnns(annIds)
    
    height = img['height']
    width = img['width']
    
    prev_center = list(list())
    segmentations = list()
    
    for j in range(len(anns)):
        if anns[j]['num_keypoints'] < 5 or anns[j]['area'] < 32*32:
            segmentations.append(anns[j]['segmentation'])
            
    for j in range(len(anns)):
        
        ## remove those persons whose keypoints are too small
        if anns[j]['num_keypoints'] < 5 or anns[j]['area'] < 32*32:
            continue
        
        person_center = [anns[j]['bbox'][0] + anns[j]['bbox'][2]/2,
                         anns[j]['bbox'][1] + anns[j]['bbox'][3]/2]
        flag = 0
        isValidation = 0 
        
        for k in range(len(prev_center)):
            dist1 = prev_center[k][0] - person_center[0]
            dist2 = prev_center[k][1] - person_center[1]
            #print dist1, dist2
            if dist1*dist1+dist2*dist2 < prev_center[k][2]*0.3:
                flag = 1
                continue
                
        currentjoin={'isValidation': isValidation,
                     'img_paths': trainimagepath + img['file_name'],
                     'objpos': person_center,
                     'image_id': img['id'],
                     'bbox': anns[j]['bbox'],
                     'img_width': width,
                     'img_height': height,
                     'segment_area': anns[j]['area'],
                     'num_keypoints': anns[j]['num_keypoints'],
                     'joint_self': np.zeros((17,3)).tolist(),
                     'scale_provided': anns[j]['bbox'][3]/368.0,
                     'segmentations': segmentations,
                     'joint_others': {},
                     'annolist_index': i ,
                     'people_index': j,
                     'numOtherPeople':0,
                     'scale_provided_other':{},
                     'objpos_other':{},
                     'bbox_other':{},
                     'segment_area_other':{},
                     'num_keypoints_other':{}
                    }    
        
        for part in range(17):
            currentjoin['joint_self'][part][0] = anns[j]['keypoints'][part*3]
            currentjoin['joint_self'][part][1] = anns[j]['keypoints'][part*3+1]
            # 2 means cropped, 0 means occluded by still on image
            if(anns[j]['keypoints'][part*3+2] == 2):
                currentjoin['joint_self'][part][2] = 1
            elif(anns[j]['keypoints'][part*3+2] == 1):
                currentjoin['joint_self'][part][2] = 0                
            else:
                currentjoin['joint_self'][part][2] = 2
            
        count_other = 1     
        currentjoin['joint_others'] ={}
       
        for k in range(len(anns)):
            
            if k==j or anns[k]['num_keypoints']==0:
                continue
                
            annop = anns[k]
            currentjoin['scale_provided_other'][count_other] = annop['bbox'][3]/368
            currentjoin['objpos_other'][count_other] = [annop['bbox'][0]+annop['bbox'][2]/2, 
                                        annop['bbox'][1]+annop['bbox'][3]/2]
            currentjoin['bbox_other'][count_other] = annop['bbox']
            currentjoin['segment_area_other'][count_other] = annop['area']
            currentjoin['num_keypoints_other'][count_other] = annop['num_keypoints']
            currentjoin['joint_others'][count_other] = np.zeros((17,3)).tolist()
            
            for part in range(17):
                currentjoin['joint_others'][count_other][part][0] = annop['keypoints'][part*3]
                currentjoin['joint_others'][count_other][part][1] = annop['keypoints'][part*3+1]
                
                if(annop['keypoints'][part*3+2] == 2):
                    currentjoin['joint_others'][count_other][part][2] = 1
                elif(annop['keypoints'][part*3+2] == 1):
                    currentjoin['joint_others'][count_other][part][2] = 0
                else:
                    currentjoin['joint_others'][count_other][part][2] = 2
                
              
            currentjoin['numOtherPeople'] = len(currentjoin['joint_others']) 
            count_other = count_other + 1
                        
        prev_center.append([person_center[0], person_center[1],
                            max(anns[j]['bbox'][2], anns[j]['bbox'][3])])
        
        joint_all[count] = currentjoin

        count = count + 1

## Validation dataset 

In [7]:
valimagepath = '/data/guest_users/liangdong/liangdong/practice_demo/val2014/'
valFlie = '/data/datasets/COCO/person_keypoints_trainval2014/person_keypoints_val2014.json'

In [8]:
coco = COCO(valFlie)
cats = coco.loadCats(coco.getCatIds())
catIds = coco.getCatIds(catNms=['person'])
imgIds = coco.getImgIds(catIds=catIds )

loading annotations into memory...
Done (t=3.67s)
creating index...
index created!


In [13]:
len(imgIds)

21634

In [9]:
numimgs = len(imgIds)

for i in range(0, numimgs):
    #print('----image: '+str(imgIds[i])+' ---------------')
    img = coco.loadImgs(imgIds[i])[0]
    annIds = coco.getAnnIds(imgIds=img['id'], catIds=catIds, iscrowd=None)
    anns = coco.loadAnns(annIds)
    
    height = img['height']
    width = img['width']
    
    prev_center = list(list())
    segmentations = list()
    
    for j in range(len(anns)):
        if anns[j]['num_keypoints'] < 5 or anns[j]['area'] < 32*32:
            segmentations.append(anns[j]['segmentation'])
            
    for j in range(len(anns)):
        
        ## remove those persons whose keypoints are too small
        if anns[j]['num_keypoints'] < 5 or anns[j]['area'] < 32*32:
            continue
        
        person_center = [anns[j]['bbox'][0] + anns[j]['bbox'][2]/2,
                         anns[j]['bbox'][1] + anns[j]['bbox'][3]/2]
        flag = 0
        isValidation = 0 
        
        for k in range(len(prev_center)):
            dist1 = prev_center[k][0] - person_center[0]
            dist2 = prev_center[k][1] - person_center[1]
            #print dist1, dist2
            if dist1*dist1+dist2*dist2 < prev_center[k][2]*0.3:
                flag = 1
                continue
                
        currentjoin={'isValidation': isValidation,
                     'img_paths': valimagepath + img['file_name'],
                     'objpos': person_center,
                     'image_id': img['id'],
                     'bbox': anns[j]['bbox'],
                     'img_width': width,
                     'img_height': height,
                     'segment_area': anns[j]['area'],
                     'num_keypoints': anns[j]['num_keypoints'],
                     'joint_self': np.zeros((17,3)).tolist(),
                     'scale_provided': anns[j]['bbox'][3]/368.0,
                     'segmentations': segmentations,
                     'joint_others': {},
                     'annolist_index': i ,
                     'people_index': j,
                     'numOtherPeople':0,
                     'scale_provided_other':{},
                     'objpos_other':{},
                     'bbox_other':{},
                     'segment_area_other':{},
                     'num_keypoints_other':{}
                    }    
        
        for part in range(17):
            currentjoin['joint_self'][part][0] = anns[j]['keypoints'][part*3]
            currentjoin['joint_self'][part][1] = anns[j]['keypoints'][part*3+1]
            # 2 means cropped, 0 means occluded by still on image
            if(anns[j]['keypoints'][part*3+2] == 2):
                currentjoin['joint_self'][part][2] = 1
            elif(anns[j]['keypoints'][part*3+2] == 1):
                currentjoin['joint_self'][part][2] = 0                
            else:
                currentjoin['joint_self'][part][2] = 2
            
        count_other = 1     
        currentjoin['joint_others'] ={}
       
        for k in range(len(anns)):
            
            if k==j or anns[k]['num_keypoints']==0:
                continue
                
            annop = anns[k]
            currentjoin['scale_provided_other'][count_other] = annop['bbox'][3]/368
            currentjoin['objpos_other'][count_other] = [annop['bbox'][0]+annop['bbox'][2]/2, 
                                        annop['bbox'][1]+annop['bbox'][3]/2]
            currentjoin['bbox_other'][count_other] = annop['bbox']
            currentjoin['segment_area_other'][count_other] = annop['area']
            currentjoin['num_keypoints_other'][count_other] = annop['num_keypoints']
            currentjoin['joint_others'][count_other] = np.zeros((17,3)).tolist()
            
            for part in range(17):
                currentjoin['joint_others'][count_other][part][0] = annop['keypoints'][part*3]
                currentjoin['joint_others'][count_other][part][1] = annop['keypoints'][part*3+1]
                
                if(annop['keypoints'][part*3+2] == 2):
                    currentjoin['joint_others'][count_other][part][2] = 1
                elif(annop['keypoints'][part*3+2] == 1):
                    currentjoin['joint_others'][count_other][part][2] = 0
                else:
                    currentjoin['joint_others'][count_other][part][2] = 2
                
              
            currentjoin['numOtherPeople'] = len(currentjoin['joint_others']) 
            count_other = count_other + 1
                        
        prev_center.append([person_center[0], person_center[1],
                            max(anns[j]['bbox'][2], anns[j]['bbox'][3])])
        
        joint_all[count] = currentjoin

        count = count + 1

## all joints length

In [10]:
print len(joint_all)

139861


## Write json file

In [ ]:
with open('trainval.json', 'w') as f:
     json.dump(joint_all, f)

## Reading data back

In [ ]:
with open('trainval.json', 'r') as f:
     data = json.load(f)